# 도서 정보 추출
- title 제목
- author 저자
- pubdate 출판일
- description 서펑 요약
- isbn13 13자리 ISBN
- itemid 알라딘 도서 목록id
- publisher 출판사
- fulldescription 알라딘 제공 서평
- categoryname 카테고리
- itempage 페이지 수
- toc 목차

In [ ]:
# 경영경제 카테고리 도서 정보 추출
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re

# 에러방지 코드
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


TTBKey = '###'

itemid_df = pd.read_csv('./chack-it-out-2/item_id.csv')
itemid_df.columns = ['index','itemid']
itemid_df.drop('index',axis=1,inplace=True)
itemid = itemid_df['itemid']

text_list = []

i=1

for item in itemid:


  OptResult_array = 'previewImgList,fulldescription,Toc,categoryIdList'
  product_lookup_url = f'http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={TTBKey}&itemIdType=ItemId&ItemId={item}&output=xml&Version=20131101&OptResult={OptResult_array}'
  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

  try:
    s = requests.Session()
    s.headers.update(headers)

    resp = requests_retry_session(session=s).get(product_lookup_url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    i=i+1
    if i%2000 == 0:
      print(i)

    title = soup.find_all('title')[-1].text
    author = soup.find('author').text
    pubdate = soup.find_all('pubdate')[1].text
    description = soup.find('description').text
    isbn13 = soup.find('isbn13').text
    itemid = soup.select('item')[0]['itemid']
    publisher = soup.find('publisher').text
    fulldescription = soup.find('fulldescription').text
    categoryname = soup.find('categoryname').text
    itempage = soup.find('itempage').text
    toc = soup.find('toc').text

    # 목차 1차 정제
    toc = toc.replace('<BR>\r\n',' ').replace('<B>',' ').replace('</B>',' ').replace('<p>',' ').replace('</p>',' ')
    toc = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]", " ", toc)
    toc = re.sub(r"\s+", " ", toc).strip()


    try:
      previewimg = soup.find('previewimglist').find('previewimg').text
    except:
      previewimg = '표지 없음'


    paprams = [title,author,pubdate,description,isbn13,itemid,publisher,fulldescription,categoryname,itempage,toc,previewimg]
    text_list.append(paprams)

  except Exception as ex:
    print(f"exception occured : {ex}")
    continue


df = pd.DataFrame(text_list, columns = ['title','author','pubdate','description','isbn13','itemid','publisher','fulldescription','categoryname','itempage','toc','previewimg'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72850 entries, 0 to 72849
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            72850 non-null  object
 1   author           72850 non-null  object
 2   pubdate          72850 non-null  object
 3   description      72850 non-null  object
 4   isbn13           72850 non-null  object
 5   itemid           72850 non-null  object
 6   publisher        72850 non-null  object
 7   fulldescription  72850 non-null  object
 8   categoryname     72850 non-null  object
 9   itempage         72850 non-null  object
 10  toc              72850 non-null  object
 11  previewimg       72850 non-null  object
dtypes: object(12)
memory usage: 6.7+ MB


In [ ]:
df.to_csv('./pre_toc_me.csv',index=False, encoding='utf-8-sig')

In [ ]:
import pickle

with open("pre_toc_me.pickle","wb") as f:
    pickle.dump(df, f)

with open("pre_toc_me.pickle","rb") as fi:
    test = pickle.load(fi)

In [ ]:
# 자기계발 카테고리 도서 추출
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re

# 에러 방지 코드
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


TTBKey = '###'

itemid_df = pd.read_csv('/Users/leeseohyun/Documents/GitHub/chack-it-out-2/self_improvement_id.csv')
itemid_df.columns = ['index','itemid']
itemid_df.drop('index',axis=1,inplace=True)
itemid = itemid_df['itemid']

text_list = []

i=1

for item in itemid:


  OptResult_array = 'previewImgList,fulldescription,Toc,categoryIdList'
  product_lookup_url = f'http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={TTBKey}&itemIdType=ItemId&ItemId={item}&output=xml&Version=20131101&OptResult={OptResult_array}'
  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

  try:
    s = requests.Session()
    s.headers.update(headers)

    resp = requests_retry_session(session=s).get(product_lookup_url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    i=i+1
    if i%2000 == 0:
      print(i)

    title = soup.find_all('title')[-1].text
    author = soup.find('author').text
    pubdate = soup.find_all('pubdate')[1].text
    description = soup.find('description').text
    isbn13 = soup.find('isbn13').text
    itemid = soup.select('item')[0]['itemid']
    publisher = soup.find('publisher').text
    fulldescription = soup.find('fulldescription').text
    categoryname = soup.find('categoryname').text
    itempage = soup.find('itempage').text
    toc = soup.find('toc').text

    # 목차 1차 정제
    toc = toc.replace('<BR>\r\n',' ').replace('<B>',' ').replace('</B>',' ').replace('<p>',' ').replace('</p>',' ')
    toc = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]", " ", toc)
    toc = re.sub(r"\s+", " ", toc).strip()


    try:
      previewimg = soup.find('previewimglist').find('previewimg').text
    except:
      previewimg = '표지 없음'


    paprams = [title,author,pubdate,description,isbn13,itemid,publisher,fulldescription,categoryname,itempage,toc,previewimg]
    text_list.append(paprams)

  except Exception as ex:
    print(f"exception occured : {ex}")
    continue


df1 = pd.DataFrame(text_list, columns = ['title','author','pubdate','description','isbn13','itemid','publisher','fulldescription','categoryname','itempage','toc','previewimg'])

In [ ]:
df1.to_csv('./pre_toc_self.csv',index=False, encoding='utf-8-sig')

In [ ]:
import pickle


with open("pre_toc_self.pickle","wb") as a:
    pickle.dump(df1, a)

with open("pre_toc_self.pickle","rb") as ai:
    test1 = pickle.load(ai)

# 데이터 웨어하우스 구축을 위한 추가 정보 크롤링
- isbn 10자리 ISBN(구버전)
- pricesales 할인가
- pricestandard 정상가
- malltype
- mileage 구매시 적립 마일리지
- salespoint 알라딘 제공 점수
- adult
- fixedprice
- customerreviewrank 고객 리뷰 순위
- fulldescription2 출판사 제공 서평
- subtitle 부제목
- originaltitle 원제목
- story 줄거리
- phraselist 인용문

In [ ]:
# 경영경제

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re
from tqdm import tqdm

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


TTBKey = '###'

itemid_df = pd.read_csv('/Users/leeseohyun/Documents/GitHub/chack-it-out-2/item_id.csv')
itemid_df.columns = ['index','itemid']
itemid_df.drop('index',axis=1,inplace=True)
itemid = itemid_df['itemid']

text_list = []

for item in tqdm(itemid):

    OptResult_array = 'previewImgList,eventList,authors,reviewList,fulldescription,fulldescription2,Toc,Story,categoryIdList,mdrecommend,phraseList'
    product_lookup_url = f'http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={TTBKey}&itemIdType=ItemId&ItemId={item}&output=xml&Version=20131101&OptResult={OptResult_array}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

    try:
        s = requests.Session()
        s.headers.update(headers)

        resp = requests_retry_session(session=s).get(product_lookup_url)
        soup = BeautifulSoup(resp.content, 'html.parser')

        isbn13 = soup.find('isbn13').text
        itemid = soup.select('item')[0]['itemid']

        isbn = soup.find('isbn').text
        pricesales = soup.find('pricesales').text
        pricestandard = soup.find('pricestandard').text
        malltype = soup.find('malltype').text
        mileage = soup.find('mileage').text
        salespoint = soup.find('salespoint').text
        adult = soup.find('adult').text
        fixedprice = soup.find('fixedprice').text
        customerreviewrank = soup.find('customerreviewrank').text
        fulldescription2 = soup.find('fulldescription2').text
        subtitle = soup.find('subtitle').text
        originaltitle = soup.find('originaltitle').text

        story = soup.find('story').text
        phraselist = soup.find('phraselist').text


        try:
            authorid = soup.find('authorid').text
        except:
            authorid = ''
        try:
            authortype = soup.find('authortype').text
        except:
            authortype = ''
        try:
            authortypedesc = soup.find('authortypedesc').text
        except:
            authortypedesc = ''
        try:
            authorinfo = soup.find('authorinfo').text
        except:
            authorinfo = ''
        try:
            authorinfolink = soup.find('authorinfolink').text
        except:
            authorinfolink = ''
        try:
            authorphoto = soup.find('authorphoto').text
        except:
            authorphoto = ''


        try:
            reviewrank = soup.find('reviewrank').text
        except:
            reviewrank = soup.find('customerreviewrank').text


        try:
            writer = soup.find('writer').text
        except:
            writer = ''


        categoryid_li = []
        for i in range(len(soup.find_all('categoryid'))):
            cat = soup.find_all('categoryid')[i].text
            categoryid_li.append(cat)
        categoryname_li = []
        for i in range(len(soup.find_all('categoryname'))):
            cat_name = soup.find_all('categoryname')[i].text
            categoryname_li.append(cat_name)


        params = [isbn13,itemid,isbn,pricesales,pricestandard,malltype,mileage,salespoint,adult,fixedprice,\
                        customerreviewrank,fulldescription2,subtitle,originaltitle,story,phraselist,authorid,authortype,authortypedesc,\
                        authorinfo,authorinfolink,authorphoto,reviewrank,writer,categoryid_li,categoryname_li]
        text_list.append(params)

    except Exception as ex:
        print(f"exception occured : {ex}")
        print(soup)
        continue



df = pd.DataFrame(text_list, columns = ['isbn13','itemid','isbn','pricesales','pricestandard','malltype','mileage','salespoint','adult','fixedprice',\
                'customerreviewrank','fulldescription2','subtitle','originaltitle','story','phraselist','authorid','authortype','authortypedesc',\
                  'authorinfo','authorinfolink','authorphoto','reviewrank','writer','categoryid_li','categoryname_li'])

In [ ]:
df.to_csv('./add_col_me.csv',index=False, encoding='utf-8-sig')

In [ ]:
import pickle

with open("add_col_me.pickle","wb") as f:
    pickle.dump(df, f)

with open("add_col_me.pickle","rb") as fi:
    test = pickle.load(fi)

In [ ]:
# 자기계발

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re
from tqdm import tqdm

def requests_retry_session(
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session


TTBKey = '###'

itemid_df = pd.read_csv('/Users/leeseohyun/Documents/GitHub/chack-it-out-2/self_improvement_id.csv')
itemid_df.columns = ['index','itemid']
itemid_df.drop('index',axis=1,inplace=True)
itemid = itemid_df['itemid']

text_list = []

for item in tqdm(itemid):

    OptResult_array = 'previewImgList,eventList,authors,reviewList,fulldescription,fulldescription2,Toc,Story,categoryIdList,mdrecommend,phraseList'
    product_lookup_url = f'http://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={TTBKey}&itemIdType=ItemId&ItemId={item}&output=xml&Version=20131101&OptResult={OptResult_array}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

    try:
        s = requests.Session()
        s.headers.update(headers)

        resp = requests_retry_session(session=s).get(product_lookup_url, timeout=60)
        soup = BeautifulSoup(resp.content, 'html.parser')

        isbn13 = soup.find('isbn13').text
        itemid = soup.select('item')[0]['itemid']

        isbn = soup.find('isbn').text
        pricesales = soup.find('pricesales').text
        pricestandard = soup.find('pricestandard').text
        malltype = soup.find('malltype').text
        mileage = soup.find('mileage').text
        salespoint = soup.find('salespoint').text
        adult = soup.find('adult').text
        fixedprice = soup.find('fixedprice').text
        customerreviewrank = soup.find('customerreviewrank').text
        fulldescription2 = soup.find('fulldescription2').text
        subtitle = soup.find('subtitle').text
        originaltitle = soup.find('originaltitle').text

        story = soup.find('story').text
        phraselist = soup.find('phraselist').text

        try:
            authorid = soup.find('authorid').text
        except:
            authorid = ''
        try:
            authortype = soup.find('authortype').text
        except:
            authortype = ''
        try:
            authortypedesc = soup.find('authortypedesc').text
        except:
            authortypedesc = ''
        try:
            authorinfo = soup.find('authorinfo').text
        except:
            authorinfo = ''
        try:
            authorinfolink = soup.find('authorinfolink').text
        except:
            authorinfolink = ''
        try:
            authorphoto = soup.find('authorphoto').text
        except:
            authorphoto = ''


        try:
            reviewrank = soup.find('reviewrank').text
        except:
            reviewrank = soup.find('customerreviewrank').text


        try:
            writer = soup.find('writer').text
        except:
            writer = ''


        categoryid_li = []
        for i in range(len(soup.find_all('categoryid'))):
            cat = soup.find_all('categoryid')[i].text
            categoryid_li.append(cat)
        categoryname_li = []
        for i in range(len(soup.find_all('categoryname'))):
            cat_name = soup.find_all('categoryname')[i].text
            categoryname_li.append(cat_name)


        params = [isbn13,itemid,isbn,pricesales,pricestandard,malltype,mileage,salespoint,adult,fixedprice,\
                        customerreviewrank,fulldescription2,subtitle,originaltitle,story,phraselist,authorid,authortype,authortypedesc,\
                        authorinfo,authorinfolink,authorphoto,reviewrank,writer,categoryid_li,categoryname_li]
        text_list.append(params)

    except Exception as ex:
        print(f"exception occured : {ex}")
        print(soup)
        continue



df_self = pd.DataFrame(text_list, columns = ['isbn13','itemid','isbn','pricesales','pricestandard','malltype','mileage','salespoint','adult','fixedprice',\
                'customerreviewrank','fulldescription2','subtitle','originaltitle','story','phraselist','authorid','authortype','authortypedesc',\
                  'authorinfo','authorinfolink','authorphoto','reviewrank','writer','categoryid_li','categoryname_li'])

In [ ]:
df_self.to_csv('./add_col_self.csv',index=False, encoding='utf-8-sig')

In [ ]:
with open("add_col_self.pickle","wb") as d:
    pickle.dump(df_self, d)

with open("add_col_self.pickle","rb") as di:
    test_self = pickle.load(di)